In [1]:
import subprocess
import socket
import re
import sys

In [6]:
# === CONFIGURATION ===
SUPABASE_DOMAIN = "db.cnubrmbhdqqlwzwcnshp.supabase.co"  # Change to your actual Supabase project domain
# ======================

In [ ]:
def get_default_gateway():
    """Get the default gateway from 'ipconfig' output."""
    result = subprocess.run(["ipconfig"], capture_output=True, text=True)
    gateway_match = re.search(r"Default Gateway[^\d]*(\d+\.\d+\.\d+\.\d+)", result.stdout)
    if not gateway_match:
        print("❌ Could not detect default gateway. Are you connected to Wi-Fi?")
        sys.exit(1)
    return gateway_match.group(1)

In [7]:
def resolve_domain_to_ip(domain):
    try:
        # Get all addresses, pick the first IPv4
        addr_info = socket.getaddrinfo(domain, None)
        for result in addr_info:
            if result[0] == socket.AF_INET:  # IPv4
                return result[4][0]
        print(f"❌ No IPv4 address found for {domain}.")
        return None
    except socket.gaierror as e:
        print(f"❌ DNS resolution failed for {domain}: {e}")
        return None

In [ ]:
def resolve_domain_via_nslookup(domain):
    try:
        result = subprocess.run(["nslookup", domain], capture_output=True, text=True)
        match = re.search(r"Address:\s*([\d\.]+)", result.stdout)
        if match:
            return match.group(1)
        else:
            print(f"❌ Could not parse IP from nslookup output:\n{result.stdout}")
            return None
    except Exception as e:
        print(f"❌ nslookup failed: {e}")
        return None

# Example:
print(resolve_domain_via_nslookup("yourproject.supabase.co"))

In [8]:
def add_static_route(ip, gateway):
    """Add a persistent static route to bypass VPN for a specific IP."""
    command = ["route", "ADD", ip, "MASK", "255.255.255.255", gateway, "-p"]
    print(f"➡ Adding static route: {command}")
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ Route added successfully: {ip} -> {gateway}")
    else:
        if "file exists" in result.stdout.lower():
            print(f"ℹ Route already exists for {ip}.")
        else:
            print(f"❌ Failed to add route. Output:\n{result.stdout}")

In [ ]:
def main():
    print("🔍 Detecting default gateway...")
    gateway = get_default_gateway()
    print(f"✅ Default gateway detected: {gateway}")

    print(f"🔍 Resolving Supabase domain ({SUPABASE_DOMAIN})...")
    supabase_ip = resolve_domain_via_nslookup(SUPABASE_DOMAIN)
    print(f"✅ Supabase IP resolved: {supabase_ip}")

    print("🔍 Adding persistent static route...")
    add_static_route(supabase_ip, gateway)


if __name__ == "__main__":
    main()

In [3]:
result = subprocess.run(["ipconfig"], capture_output=True, text=True)
gateway_match = re.search(r"Default Gateway[^\d]*(\d+\.\d+\.\d+\.\d+)", result.stdout)